In [ ]:
!pip install faker

# Configure and Load the Embedding Model


### Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [ ]:
!pip install eland elasticsearch transformers sentence_transformers torch==1.11

In [ ]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

### Configure elasticsearch authentication.
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [ ]:
import getpass

In [ ]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_key = getpass.getpass('Enter cluster API key:  ')

### Connect to Elastic Cloud

In [ ]:
es = Elasticsearch(cloud_id=es_cloud_id,
#                   api_key=(es_api_id, es_api_key)
                   api_key=es_api_key
                   )
es.info() # should return cluster info

## Loading a model from Hugging Face model hub

The Elastic Stack machine learning features support transformer models that conform to the standard BERT model interface and use the WordPiece tokenization algorithm.

A current list of supported architectures can be viewed in the
[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html)


### Download an embedding model from Hugging Face using the HF copy link

For this example we will be using the [sentence-transformers/msmarco-MiniLM-L-12-v3](https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-12-v3) model


### Download the Model
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code:
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

_note and warnings about "Some weights of the model checkpoint" can be ignored_

In [ ]:
hf_model_id='dslim/bert-base-NER'
tm = TransformerModel(hf_model_id, "ner")

### Set and confirm the model ID
To make the name compatible with elasticsearch, the '/' is replaced with '__'



In [ ]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

## Export the model in a TorchScrpt representation which Elasticsearch uses

In [ ]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

## Load the model into Elasticsearch
Model should not already exist in elasticsearch

In [ ]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

## Starting the Model

### View information about the model
This is not required but can be handy to get a model overivew

In [ ]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

### Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

### Verify the model started without issue

In [ ]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

# Create the ingest pipeline

In [ ]:
description= "PII redacting ingest pipeline"

processors= [
    {
      "set": {
        "field": "redacted",
        "value": "{{{message}}}"
      }
    },
    {
      "inference": {
        "model_id": es_model_id,
        "field_map": {
          "message": "text_field"
        }
      }
    },
    {
      "script": {
        "lang": "painless",
        "source": """String msg = ctx['message'];
      for (item in ctx['ml']['inference']['entities']) {
        msg = msg.replace(item['entity'], '<' + item['class_name'] + '>')
      }
      ctx['redacted']=msg""",
        "if": "return ctx['ml']['inference']['entities'].isEmpty() == false",
        "tag": "ner_redact",
        "description": "Redact NER entities"
      }
    },
    {
      "redact": {
        "field": "redacted",
        "patterns": [
          "%{PHONE:PHONE}",
          "%{SSN:SSN}"
        ],
        "pattern_definitions": {
          "SSN": """\d{3}-?\d{2}-?\d{4}""",
          "PHONE": """\d{3}-?\d{3}-?\d{4}"""
        }
      }
    },
    {
      "remove": {
        "field": [
          "message",
          "ml"
        ]
      }
    }
  ]

on_failure= [
    {
      "set": {
        "field": "failure",
        "value": "pii_script-redact"
      }
    }
  ]



response = es.ingest.put_pipeline(id="pii_redaction_pipeline_book",
                                  description=description,
                                  processors=processors,
                                  on_failure=on_failure
)


# Print the response
print(response)

# Configure the index template
Match any indices created with the name pattern `pii_data*`
Create the mappings for the fields
Configure new data to use our `pii_redaction_pipeline` by default

In [ ]:
index_patterns =[
    "pii_data*"
  ]
order = 1
settings = {
      "number_of_shards": 1,
      "number_of_replicas": 1,
      "index.default_pipeline": "pii_redaction_pipeline_book"
    }
mappings = {
      "properties": {
        "message": {
          "type": "text"
        },
        "status": {
          "type": "keyword"
        },
        "redacted": {
          "type": "text"
        }
      }
    }


# Create the index template
response = es.indices.put_template(name="pii_book_template",
                                   index_patterns=index_patterns,
                                   order=order,
                                   settings=settings,
                                   mappings=mappings
                                   )

# Print the response
print(response)


# Generate fake PII data

In [ ]:
from faker import Faker
import json
from pprint import pprint

# Create an instance of the Faker class
fake = Faker()

# Define a function to generate fake personal identification information
def generate_fake_pii(num_records):

  fake_data = []

  for x in range(num_records):
  # Generate fake personal identification information
    fn = fake.first_name()
    ln = fake.last_name()
    pn = fake.phone_number()
    sn = fake.ssn()
    ai = fake.random_element(elements=('active', 'inactive'))

    call_log = {
        'message' : f'{fn} {ln} called in from {pn}. Their account number is {sn}',
        'status' : ai
        }

    fake_data.append(call_log)
  return fake_data

# Generate fake personal identification information for N individuals
num_records = 10 # Set the desired number of records
fake_pii_data = generate_fake_pii(num_records)

pprint(fake_pii_data)

# Ingest Fake data into Elasticsearch
Loading the data to the pii_data index will pass the data through the configured ingest pipeline redacting pii.

In [ ]:
from elasticsearch import Elasticsearch, helpers

# Define a function to generate an array of documents for Elasticsearch
def generate_documents_array(fake_data, index_name):

  documents = []

  # Iterate over the fake data and format it as documents
  for call in fake_data:
    # Create a document with the _index and _source keys
    document = {
      '_index': index_name,
      '_source': call
    }

    documents.append(document)

  return documents


# Generate the array of documents for Elasticsearch
index_name = 'pii_data-book'
documents_array = generate_documents_array(fake_pii_data, index_name)

# Convert the bulk index body to a single string with newline separators
print("Bulk request: ")
print(documents_array)

try:
	response = helpers.bulk(es, documents_array)
	print ("\nRESPONSE:", response)
except Exception as e:
	print("\nERROR:", e)

In [ ]:
query = {"match_all":{}}

response = es.search(
    index=index_name,
    query=query
    )

pprint(response['hits']['hits'])